# Agent State Machine (ASM) - Quick Start Guide

An **agent** is a wrapper around a finite state machine designed to accomplish a specific task and will be referred to as ASM.

A **StateMachineBuilder** is used to build the ASM from a manifest and a state diagram.

```mermaid
flowchart TD
    A["state_diagram"] -->|Input| B["StateMachineBuilder"]
    A2["state_manifest"] -->|Input| C["StateModel"]
    C -->|Build| B
    B --> D["fsm"]
```


---

## Example: Standard Assistant

In this example, we will demonstrate how to create a simple **Assistant** agent using a state diagram and state manifest.

### a) Define State Diagram

The following is a simple example of an **Assistant** agent using 3 states:

* **INIT:** Collect initial input.
* **GENERATE:** The state where the LLM generates a response.
* **FINAL:** Return final output.

```mermaid
stateDiagram-v2
direction LR
INIT --> GENERATE: next / action
GENERATE --> FINAL: next / action
```


In [1]:
STATE_DIAGRAM = """
    INIT --> GENERATE
    GENERATE --> FINAL
    """

### b) Define State Manifest

The state manifest is a dictionary. 
Each state in the manifest corresponds to a state in the state diagram.


In [2]:
STATE_MANIFEST_V1 = {
    "INIT": {
        "input_data": {
            "user_message": "Write a one sentence story",
            "llm_config": {"type": "getter", "dependency": "get_llm_config"}
            }
    },
    "GENERATE": {
        "module_path": "gai.asm.states",
        "class_name": "PureActionState",
        "title": "GENERATE",
        "action": "generate",
        "input_data": {
            "user_message": {"type":"state_bag","dependency":"user_message"},
            "llm_config": {"type": "state_bag", "dependency": "llm_config"},
        },
        "output_data": ["streamer", "get_assistant_message"],
    },
    "FINAL": {"output_data": ["monologue"]},
}


### c) Create state action

In [3]:
from gai.dialogue.chat import AsyncOpenAI

async def generate_action(state):
    
    llm_config = state.input["llm_config"]
    client = AsyncOpenAI(llm_config)
    
    # Get message from input
    user_message = state.input["user_message"]
    
    # Execute
    
    response = await client.chat.completions.create(
        model=llm_config["model"],
        messages=[{
            "role":"user",
            "content":user_message
            }],
        max_tokens=50,
        stream=True
    )
    
    assistant_message = ""
    async def streamer():
        nonlocal assistant_message
        async for chunk in response:
            chunk = chunk.choices[0].delta.content
            if isinstance(chunk,str) and chunk:
                assistant_message += chunk
                yield chunk

    state.machine.state_bag["get_assistant_message"] = lambda: assistant_message
    state.machine.state_bag["streamer"] = streamer()

### c) Build State Machine

In [4]:
from gai.asm import AsyncStateMachine

with AsyncStateMachine.StateMachineBuilder(STATE_DIAGRAM) as builder:
    fsm = builder.build(
        STATE_MANIFEST_V1,
        get_llm_config=lambda state: {
            "client_type": "anthropic",
            "model": "claude-opus-4-20250514",
        },
        # get_llm_config=lambda state: {
        #     "client_type": "gai",
        #     "model": "ttt",
        #     "url": "http://gai-llm-svr:12031/gen/v1/chat/completions"
        # },
        agent_name="Sara",
        generate=generate_action,
    )


### d) Run State Machine (INIT->GENERATE)

In [5]:
await fsm.run_async()
async for chunk in fsm.state_bag["streamer"]:
    print(chunk,end='',flush=True)
print("\n\n")

The old lighthouse keeper discovered that the ships he'd been guiding to safety for forty years were actually ghosts, doomed to repeat their final voyage unless someone lit the beacon each night.




### d) Continue (GENERATE->FINAL)

In [6]:
await fsm.run_async()
print("State History:")
for state in fsm.state_history:
    print(f"State: {state['state']}")
    print(f"- input: {state['input']}")
    print(f"- output: {state['output']}")
    print("-" * 20)
print("Assistant Message:")
fsm.state_bag["get_assistant_message"]()

State History:
State: INIT
- input: {'user_message': 'Write a one sentence story', 'monologue': <gai.asm.monologue.Monologue object at 0x71a17c7fd6c0>, 'step': 0, 'time': datetime.datetime(2025, 6, 14, 4, 26, 14, 916184), 'name': 'Sara', 'llm_config': {'client_type': 'anthropic', 'model': 'claude-opus-4-20250514'}}
- output: {'name': 'Sara', 'monologue': <gai.asm.monologue.Monologue object at 0x71a17de709d0>, 'step': 1, 'time': datetime.datetime(2025, 6, 14, 4, 26, 14, 916283)}
--------------------
State: GENERATE
- input: {'name': 'Sara', 'monologue': <gai.asm.monologue.Monologue object at 0x71a17c7fe4d0>, 'step': 1, 'time': datetime.datetime(2025, 6, 14, 4, 26, 14, 916488), 'user_message': 'Write a one sentence story', 'llm_config': {'client_type': 'anthropic', 'model': 'claude-opus-4-20250514'}}
- output: {'streamer': <async_generator object generate_action.<locals>.streamer at 0x71a16b3102c0>, 'get_assistant_message': <function generate_action.<locals>.<lambda> at 0x71a16b309bd0>, 

"The old lighthouse keeper discovered that the ships he'd been guiding to safety for forty years were actually ghosts, doomed to repeat their final voyage unless someone lit the beacon each night."

---

## Example: Standard Assistant (Part 2)

Same example but with an additional state to demonstrate context management by monologue messages.

```mermaid
stateDiagram-v2
direction LR
INIT --> GENERATE
GENERATE --> CONTINUE
CONTINUE --> FINAL
```


In [1]:
STATE_DIAGRAM = """
    INIT --> GENERATE
    GENERATE --> CONTINUE
    CONTINUE --> FINAL
    """
    
STATE_MANIFEST_V1 = {
    "INIT": {
        "input_data": {
            "user_message": "Write a one sentence story",
            "llm_config": {"type": "getter", "dependency": "get_llm_config"},
        }
    },
    "GENERATE": {
        "module_path": "gai.asm.states",
        "class_name": "PureActionState",
        "title": "GENERATE",
        "action": "generate_action",
        "input_data": {
            "user_message": {"type": "state_bag", "dependency": "user_message"},
            "llm_config": {"type": "state_bag", "dependency": "llm_config"},
        },
        "output_data": ["streamer", "get_assistant_message"],
    },
    "CONTINUE": {
        "module_path": "gai.asm.states",
        "class_name": "PureActionState",
        "title": "CONTINUE",
        "action": "continue_action",
        "input_data": {
            "llm_config": {"type": "state_bag", "dependency": "llm_config"},
        },
        "output_data": ["streamer", "get_assistant_message"],
    },
    "FINAL": {"output_data": ["monologue"]},
}

from gai.dialogue.chat import AsyncOpenAI

async def generate_action(state):

    llm_config = state.input["llm_config"]
    client = AsyncOpenAI(llm_config)

    # Import data from state_bag
    user_message = state.input["user_message"]
    state.machine.monologue.add_user_message(state=state,content=user_message)
    
    # Execute
    
    response = await client.chat.completions.create(
        model=llm_config["model"],
        messages=state.machine.monologue.list_chat_messages(),
        max_tokens=50,
        stream=True
    )
    
    assistant_message = ""
    async def streamer():
        nonlocal assistant_message
        async for chunk in response:
            chunk = chunk.choices[0].delta.content
            if isinstance(chunk,str) and chunk:
                assistant_message += chunk
                yield chunk
        state.machine.monologue.add_assistant_message(state=state,content=assistant_message)
        # Need to update the stale history due to delayed output
        state.machine.state_history[-1]["output"]["monologue"]=state.machine.monologue.copy()
    
    state.machine.state_bag["get_assistant_message"] = lambda: assistant_message
    state.machine.state_bag["streamer"] = streamer()

async def continue_action(state):
    
    llm_config = state.input["llm_config"]
    client = AsyncOpenAI(llm_config)
    
    # Import data from state_bag
    state.machine.monologue.add_user_message(state=state,content="Please continue.")
    
    # Execute
    
    response = await client.chat.completions.create(
        model=llm_config["model"],
        messages=state.machine.monologue.list_chat_messages(),
        max_tokens=50,
        stream=True
    )
    
    assistant_message = ""
    async def streamer():
        nonlocal assistant_message
        async for chunk in response:
            chunk = chunk.choices[0].delta.content
            if isinstance(chunk,str) and chunk:
                assistant_message += chunk
                yield chunk
        state.machine.monologue.add_assistant_message(
            state=state, content=assistant_message
        )
        # Need to update the stale history due to delayed output
        state.machine.state_history[-1]["output"]["monologue"] = (
            state.machine.monologue.copy()
        )
    
    state.machine.state_bag["get_assistant_message"] = lambda: assistant_message
    state.machine.state_bag["streamer"] = streamer()
    
from gai.asm import AsyncStateMachine

with AsyncStateMachine.StateMachineBuilder(STATE_DIAGRAM) as builder:
    fsm = builder.build(
        STATE_MANIFEST_V1,
        get_llm_config=lambda state: {
            "client_type": "anthropic",
            "model": "claude-opus-4-20250514",
        },
        # get_llm_config=lambda state: {
        #     "client_type": "gai",
        #     "model": "ttt",
        #     "url": "http://gai-llm-svr:12031/gen/v1/chat/completions"
        # },
        agent_name="Sara",
        generate_action=generate_action,
        continue_action=continue_action,
    )


### d) Run State Machine (INIT->GENERATE)

In [2]:
await fsm.run_async()
async for chunk in fsm.state_bag["streamer"]:
    print(chunk,end='',flush=True)
print("\n\n")

print("Step 1:")
print("Before:")
for message in fsm.state_history[1]["input"]["monologue"].list_messages():
    print(
        f"{message.header.timestamp} {message.header.sender} > {message.body.content}"
    )

print("After:")
for message in fsm.state_history[1]["output"]["monologue"].list_messages():
    print(
        f"{message.header.timestamp} {message.header.sender} > {message.body.content}"
    )

The old lighthouse keeper discovered that the mysterious flashes he'd been answering for forty years were not from ships, but from another lighthouse keeper on a distant shore, equally alone, speaking in light.


Step 1:
Before:
After:
1749875293.8218691 User > Write a one sentence story
1749875299.4398446 Sara > The old lighthouse keeper discovered that the mysterious flashes he'd been answering for forty years were not from ships, but from another lighthouse keeper on a distant shore, equally alone, speaking in light.


### c) Run State Machine (GENERATE->CONTINUE)

In [3]:
await fsm.run_async()
async for chunk in fsm.state_bag["streamer"]:
    print(chunk,end='',flush=True)
print("\n\n")

print("Step 2:")
print("Before:")
for message in fsm.state_history[2]["input"]["monologue"].list_messages():
    print(
        f"{message.header.timestamp} {message.header.sender} > {message.body.content}"
    )

print("After:")
for message in fsm.state_history[2]["output"]["monologue"].list_messages():
    print(
        f"{message.header.timestamp} {message.header.sender} > {message.body.content}"
    )

When the coast guard arrived that morning to tell him the lighthouse was being decommissioned, he frantically flashed his final message across the dark waters—"I am here, I have always been here"—and for the first time in


Step 2:
Before:
1749875293.8218691 User > Write a one sentence story
1749875299.4398446 Sara > The old lighthouse keeper discovered that the mysterious flashes he'd been answering for forty years were not from ships, but from another lighthouse keeper on a distant shore, equally alone, speaking in light.
After:
1749875293.8218691 User > Write a one sentence story
1749875299.4398446 Sara > The old lighthouse keeper discovered that the mysterious flashes he'd been answering for forty years were not from ships, but from another lighthouse keeper on a distant shore, equally alone, speaking in light.
1749875308.0710979 User > Please continue.
1749875337.486953 Sara > When the coast guard arrived that morning to tell him the lighthouse was being decommissioned, he frantic

### e) END (GENERATE->FINAL)

In [10]:
await fsm.run_async()
print("State History:")
for state in fsm.state_history:
    print(f"State: {state['state']}")
    print(f"- input: {state['input']}")
    print(f"- output: {state['output']}")
    print("-" * 20)
print("Assistant Message:")
fsm.state_bag["get_assistant_message"]()

State History:
State: INIT
- input: {'user_message': 'Write a one sentence story', 'monologue': <gai.asm.monologue.Monologue object at 0x71a16b3760b0>, 'step': 0, 'time': datetime.datetime(2025, 6, 14, 4, 26, 53, 691371), 'name': 'Sara', 'llm_config': {'client_type': 'anthropic', 'model': 'claude-opus-4-20250514'}}
- output: {'name': 'Sara', 'monologue': <gai.asm.monologue.Monologue object at 0x71a16b375780>, 'step': 1, 'time': datetime.datetime(2025, 6, 14, 4, 26, 53, 691447)}
--------------------
State: GENERATE
- input: {'name': 'Sara', 'monologue': <gai.asm.monologue.Monologue object at 0x71a16b3761a0>, 'step': 1, 'time': datetime.datetime(2025, 6, 14, 4, 26, 53, 691701), 'user_message': 'Write a one sentence story', 'llm_config': {'client_type': 'anthropic', 'model': 'claude-opus-4-20250514'}}
- output: {'streamer': <async_generator object generate_action.<locals>.streamer at 0x71a16b311f40>, 'get_assistant_message': <function generate_action.<locals>.<lambda> at 0x71a16b30ac20>, 

"As dawn broke over the harbor, Thomas set down his cup of cold coffee and watched the translucent hull of a schooner fade into the morning mist, just as it had every Tuesday since 1847. He'd figured out the"

---

## Example: ToolUse Assistant using Anthropic models

```mermaid
stateDiagram-v2
direction LR
INIT --> TOOL_CALL
TOOL_CALL --> TOOL_USE
TOOL_USE --> FINAL
```



In [1]:
from gai.asm import AsyncStateMachine, FileMonologue
from gai.mcp.client.mcp_client import McpAggregatedClient
monologue = FileMonologue()
monologue.reset()

builder = AsyncStateMachine.StateMachineBuilder(
    """
    INIT --> TOOL_CALL
    TOOL_CALL--> TOOL_USE
    TOOL_USE --> FINAL
    """
)    
fsm = builder.build(
    {
        "INIT": {
            "input_data": {
                "user_message": "What time is it now?",
                "llm_config": {"type": "getter", "dependency": "get_llm_config"},
                "mcp_client": {"type": "getter", "dependency": "get_mcp_client"},
            }
        },
        "TOOL_CALL": {
            "module_path": "gai.asm.states",
            "class_name": "AnthropicToolCallState",
            "title": "TOOL_CALL",
            "input_data": {
                "user_message": {"type": "state_bag", "dependency": "user_message"},
                "llm_config": {"type": "state_bag", "dependency": "llm_config"},
                "mcp_client": {"type": "state_bag", "dependency": "mcp_client"},
            },
            "output_data": ["streamer", "get_assistant_message"],
        },
        "TOOL_USE": {
            "module_path": "gai.asm.states",
            "class_name": "AnthropicToolUseState",
            "title": "TOOL_USE",
            "input_data": {
                "llm_config": {"type": "state_bag", "dependency": "llm_config"},
                "mcp_client": {"type": "state_bag", "dependency": "mcp_client"},
            },
            "output_data": ["streamer", "get_assistant_message"],
        },
        "FINAL": {"output_data": ["monologue"]},
    },
    get_llm_config=lambda state: {
        "client_type": "anthropic",
        # "model": "claude-opus-4-20250514",
        "model": "claude-sonnet-4-20250514",
        "max_tokens": 32000,
        "temperature": 0.7,
        "top_p": 0.95,
        "tools": True,
    },
    get_mcp_client=lambda state: McpAggregatedClient(["mcp-time"]),  # Replace with actual MCP client if needed
    monologue=monologue,
)

### a) Generate Tool Call (INIT -> TOOL_CALL)

In [2]:
await fsm.run_async()
async for chunk in fsm.state_bag["streamer"]:
    if isinstance(chunk, str):
        print(chunk, end="", flush=True)
print("\n\n")
for message in fsm.state_history[-1]["output"]["monologue"].list_messages():
    print(
        f"{message.header.timestamp} {message.header.sender} > {message.body.content}"
    )




1750071176.122163 User > What time is it now?
1750071179.683299 Assistant > [{'id': 'toolu_01NSErGxSbMH1xp4oT5qUUpV', 'input': {'format': 'YYYY-MM-DD HH:mm:ss'}, 'name': 'current_time', 'type': 'tool_use'}]


### b) Use Tool (TOOL_CALL -> TOOL_USE)

In [3]:
await fsm.run_async()
async for chunk in fsm.state_bag["streamer"]:
    if isinstance(chunk, str):
        print(chunk, end="", flush=True)
print("\n\n")
for message in fsm.state_history[-1]["output"]["monologue"].list_messages():
    print(
        f"{message.header.timestamp} {message.header.sender} > {message.body.content}"
    )

The current time is **2025-06-16 10:53:02 UTC**.


1750071176.122163 User > What time is it now?
1750071179.683299 Assistant > [{'id': 'toolu_01NSErGxSbMH1xp4oT5qUUpV', 'input': {'format': 'YYYY-MM-DD HH:mm:ss'}, 'name': 'current_time', 'type': 'tool_use'}]
1750071182.448625 User > [{'type': 'tool_result', 'tool_use_id': 'toolu_01NSErGxSbMH1xp4oT5qUUpV', 'content': 'Current UTC time is 2025-06-16 10:53:02, and the time in UTC is 2025-06-16 10:53:02.'}]
1750071184.4425583 Assistant > [{'citations': None, 'text': 'The current time is **2025-06-16 10:53:02 UTC**.', 'type': 'text'}]
